# 01 — Data Exploration

Load 6 asset class NAV series from `funds.db` and explore their basic properties.

**Asset classes (Direct Growth, 2013–2025)**
| ID | Label | Scheme Code |
|---|---|---|
| `equity` | Passive Equity (Nifty 50) | 120716 |
| `gold` | Gold ETF | 111954 |
| `gilt` | Government Bonds | 119116 |
| `corp_bond` | Corporate Bonds | 118987 |
| `short_dur` | Short Duration Debt | 118780 |
| `liquid` | Liquid (Cash Proxy) | 119568 |

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid', palette='tab10')
plt.rcParams['figure.dpi'] = 120

DB_PATH = Path('../funds.db')
assert DB_PATH.exists(), f'DB not found at {DB_PATH.resolve()}'
print('DB found:', DB_PATH.resolve())

In [ ]:
ASSETS = {
    'equity':    {'scheme_code': 120716, 'label': 'Passive Equity (Nifty 50)'},
    'gold':      {'scheme_code': 111954, 'label': 'Gold ETF'},
    'gilt':      {'scheme_code': 119116, 'label': 'Government Bonds'},
    'corp_bond': {'scheme_code': 118987, 'label': 'Corporate Bonds'},
    'short_dur': {'scheme_code': 118780, 'label': 'Short Duration Debt'},
    'liquid':    {'scheme_code': 119568, 'label': 'Liquid (Cash Proxy)'},
}

START_DATE = '2013-01-01'
END_DATE   = '2025-12-31'

conn = sqlite3.connect(DB_PATH)
nav_frames = {}
for asset_id, meta in ASSETS.items():
    df = pd.read_sql_query(
        'SELECT date, nav FROM nav WHERE scheme_code = ? AND date >= ? AND date <= ? ORDER BY date',
        conn,
        params=(meta['scheme_code'], START_DATE, END_DATE),
        parse_dates=['date'],
        index_col='date',
    )
    nav_frames[asset_id] = df['nav'].rename(asset_id)
    print(f"{asset_id:12s}  rows={len(df):5d}  {df.index.min().date()} to {df.index.max().date()}")
conn.close()

In [ ]:
# Combine into daily DataFrame and forward-fill weekend/holiday gaps
nav_df = pd.DataFrame(nav_frames).sort_index()
nav_df = nav_df.ffill()

# Resample to month-end
monthly_nav = nav_df.resample('ME').last().dropna()
print(f'Monthly observations: {len(monthly_nav)} months')
print(f'Common period: {monthly_nav.index[0].date()} to {monthly_nav.index[-1].date()}')
monthly_nav.head()

In [ ]:
# Normalise all series to 100 at common start
norm = (monthly_nav / monthly_nav.iloc[0]) * 100

fig, ax = plt.subplots(figsize=(13, 5))
colors = ['#3B82F6','#F59E0B','#10B981','#6366F1','#EC4899','#8B5CF6']
for i, col in enumerate(norm.columns):
    ax.plot(norm.index, norm[col], label=ASSETS[col]['label'], color=colors[i], linewidth=1.8)

ax.set_title('Asset Class Growth (Base = 100 at Jan 2013)', fontsize=14, fontweight='bold')
ax.set_ylabel('Indexed NAV')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:.0f}'))
ax.legend(loc='upper left', fontsize=9)
fig.tight_layout()
plt.show()

In [ ]:
# Compute monthly returns
monthly_ret = monthly_nav.pct_change().dropna()

# Annualised statistics
n_months = len(monthly_ret)
ann_ret  = (1 + monthly_ret).prod() ** (12 / n_months) - 1
ann_vol  = monthly_ret.std() * np.sqrt(12)
sharpe   = (ann_ret - 0.065) / ann_vol
skew     = monthly_ret.skew()
kurt     = monthly_ret.kurt()

stats = pd.DataFrame({
    'Label':          {k: v['label'] for k, v in ASSETS.items()},
    'Ann Return %':   (ann_ret * 100).round(2),
    'Ann Vol %':      (ann_vol * 100).round(2),
    'Sharpe (6.5%)':  sharpe.round(3),
    'Skewness':       skew.round(3),
    'Kurtosis':       kurt.round(3),
}).sort_values('Sharpe (6.5%)', ascending=False)

print('\nAnnualised Statistics (2013–2025)')
stats

In [ ]:
# Return distribution histograms
fig, axes = plt.subplots(2, 3, figsize=(13, 7))
axes = axes.flatten()

for i, (col, color) in enumerate(zip(monthly_ret.columns, colors)):
    ax = axes[i]
    monthly_ret[col].hist(bins=30, ax=ax, color=color, alpha=0.75, edgecolor='white')
    ax.axvline(0, color='gray', linestyle='--', linewidth=0.8)
    ax.set_title(ASSETS[col]['label'], fontsize=10)
    ax.set_xlabel('Monthly Return')
    ax.xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))

fig.suptitle('Monthly Return Distributions (2013–2025)', fontsize=13, fontweight='bold', y=1.01)
fig.tight_layout()
plt.show()

In [ ]:
# Calendar-year heatmap of annual returns
annual_ret = (monthly_nav.resample('YE').last().pct_change().dropna() * 100).round(1)
annual_ret.index = annual_ret.index.year
annual_ret.columns = [ASSETS[c]['label'].split('(')[0].strip() for c in annual_ret.columns]

fig, ax = plt.subplots(figsize=(13, 6))
sns.heatmap(
    annual_ret.T,
    annot=True, fmt='.1f',
    cmap='RdYlGn', center=0, vmin=-30, vmax=50,
    linewidths=0.5, linecolor='white',
    ax=ax,
    cbar_kws={'label': 'Annual Return %'},
)
ax.set_title('Calendar-Year Returns by Asset Class (%)', fontsize=13, fontweight='bold')
ax.set_xlabel('Year')
fig.tight_layout()
plt.show()